In [107]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
%matplotlib inline
from tensorflow import keras



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
data = []

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        data.append(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
from pathlib import Path

from plotnine import *
import os
# suppress tf informational and warning messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras import backend as K

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import preprocessing
from sklearn import metrics

import category_encoders as ce


%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_STATE = 2112
keras.utils.set_random_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
INPUT_DIR = Path('/kaggle/input/playground-series-s4e7')
TARGET_NAME = 'Response'

train_data = pd.read_csv(INPUT_DIR / 'train.csv')
test_data = pd.read_csv(INPUT_DIR / 'test.csv')

cont_features = []    
cat_features = [
    'Gender', 'Driving_License', 'Previously_Insured', 
    'Vehicle_Age', 'Vehicle_Damage', 
    'Policy_Sales_Channel', 'Region_Code', 'Age', 'Vintage', 'Annual_Premium']

def fe(df):
    return df

# convert to pipeline:
all_data = fe(pd.concat([test_data, train_data]))

oe = preprocessing.OrdinalEncoder()
all_data[cat_features] = oe.fit_transform(all_data.filter(cat_features)).astype('int')

cat_features_card = {}
for f in cat_features:
    cat_features_card[f] = 1 + all_data[f].max()


# treat the 0-1 features as continuous
# everything else will go into an embedding layer.
cont_features = [
    'Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Damage']
cat_features = [ 
    'Vehicle_Age', 'Policy_Sales_Channel', 
    'Region_Code', 'Age', 'Vintage', 'Annual_Premium']

train_data = all_data.query(f"not {TARGET_NAME}.isna()")
test_data  = all_data.query(f"{TARGET_NAME}.isna()").drop(columns=[TARGET_NAME])
features = cont_features + cat_features

# tidy up
all_data = None
import gc
gc.collect()

28

In [4]:

df_train = train_data.drop(columns =['id'])
df_test= test_data.drop(columns= ['id'])

In [108]:
df_train = pd.read_csv(data[1])
df_test = pd.read_csv(data[2])
df_train = df_train.drop(columns =['id'])
df_test= df_test.drop(columns= ['id'])

In [109]:
def function1(t):
    if t =='Female':
        return 0
    elif t == 'Male':
        return 1
    else:
        return None

def function2(t):
    if t == '1-2 Year':
        return 1
    elif t == '> 2 Years':
        return 3
    elif '< 1 Year':
        return 0
    else:
        return None
    
def function3(t):
    if t =='No':
        return 0
    elif t == 'Yes':
        return 1
    else:
        return None
    
trans1 = [function1(t) for t in df_train['Gender']]
trans2 = [function2(t) for t in df_train['Vehicle_Age']]
trans3 = [function3(t) for t in df_train['Vehicle_Damage']]

df_train['Gender'] = pd.Series(data=trans1)
df_train['Vehicle_Age'] = pd.Series(data=trans2)
df_train['Vehicle_Damage'] = pd.Series(data=trans3)

trans1 = [function1(t) for t in df_test['Gender']]
trans2 = [function2(t) for t in df_test['Vehicle_Age']]
trans3 = [function3(t) for t in df_test['Vehicle_Damage']]

df_test['Gender'] = pd.Series(data=trans1)
df_test['Vehicle_Age'] = pd.Series(data=trans2)
df_test['Vehicle_Damage'] = pd.Series(data=trans3)

In [5]:
# Age는 log로 돌리고
# Region_code - 별로 안중요한거 같음
# Driving Licence -도 안중요한거 같은 because driver license가 있어서야지 운전을 할수 있고-> 
# Distibution_premium - 1, 3 분위수로 지워봄.
# Distibution of vintage는 그대로 냅둬도 될거 같음
# Distibution of policy_sales_channel 그래도 냅두자

In [96]:
# Q1 = df_train['Annual_Premium'].quantile(0.25)
# Q3 = df_train['Annual_Premium'].quantile(0.75)
# IQR = Q3 - Q1
# df_train = df_train[~((df_train['Annual_Premium'] < (Q1 - 1.5 * IQR)) | (df_train['Annual_Premium'] > (Q3 + 1.5 * IQR)))]
# #이렇게 제거하면 35.49724781793551가 제거됨

In [7]:
# df_train = df_train.drop(columns=['Driving_License'])
# df_test = df_test.drop(columns=['Driving_License'])

In [110]:
df_train['Age']= np.log1p(df_train['Age'])
df_test['Age']= np.log1p(df_test['Age'])


In [123]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Gender,11504798.0,0.541351,0.498287,0.000000,0.000000,1.000000,1.000000,1.000000
Age,11504798.0,3.603018,0.373317,3.044522,3.218876,3.610918,3.912023,4.454347
Driving_License,11504798.0,0.998022,0.044431,0.000000,1.000000,1.000000,1.000000,1.000000
Region_Code,11504798.0,26.418690,12.991590,0.000000,15.000000,28.000000,35.000000,52.000000
Previously_Insured,11504798.0,0.462997,0.498629,0.000000,0.000000,0.000000,1.000000,1.000000
Vehicle_Age,11504798.0,0.644653,0.691628,0.000000,0.000000,1.000000,1.000000,3.000000
Vehicle_Damage,11504798.0,0.502680,0.499993,0.000000,0.000000,1.000000,1.000000,1.000000
Annual_Premium,11504798.0,30461.370411,16454.745205,2630.000000,25277.000000,31824.000000,39451.000000,540165.000000
Policy_Sales_Channel,11504798.0,112.425442,54.035708,1.000000,29.000000,151.000000,152.000000,163.000000
Vintage,11504798.0,163.897744,79.979531,10.000000,99.000000,166.000000,232.000000,299.000000


In [51]:
df_train = pd.get_dummies(df_train, columns=["Gender", "Vehicle_Age", "Vehicle_Damage"]).astype(int)
df_test = pd.get_dummies(df_test, columns=["Gender", "Vehicle_Age", "Vehicle_Damage"]).astype(int)

In [ ]:
# Previously_Insured,Region_Code,Policy_Sales_Channel,Gender_Female,Gender_Male,Vehicle_Age_1-2 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_No,Vehicle_Damage_Yes

In [ ]:
# Age,Annual_Premium,Vintage

In [6]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X = df_train.loc[:, df_train.columns != "Response"]
y = df_train['Response']


X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.005, random_state=42)

print("Train set size:", X_train.shape)
print("Dev set size:", X_dev.shape)
# print("Test set size:", X_test.shape)

Train set size: (11447274, 10)
Dev set size: (57524, 10)


In [54]:
cat_features_names= ['Previously_Insured','Driving_License','Region_Code','Policy_Sales_Channel','Gender_Female','Gender_Male','Vehicle_Age_1-2 Year','Vehicle_Age_< 1 Year','Vehicle_Age_> 2 Years','Vehicle_Damage_No','Vehicle_Damage_Yes']
cont_features_names = ['Age','Annual_Premium','Vintage']

In [ ]:
cat_features_names= ['Previously_Insured','Driving_License','Region_Code','Policy_Sales_Channel','Gender','Vehicle_Age','Vehicle_Damage']
cont_features_names = ['Age','Annual_Premium','Vintage']

In [10]:
cat_features_names= ['Previously_Insured','Driving_License','Region_Code','Policy_Sales_Channel','Gender','Vehicle_Age','Vehicle_Damage']
cont_features_names = ['Age','Annual_Premium','Vintage']
# Creating TensorFlow datasets for categorical features individually
cat_features_train = {f: tf.data.Dataset.from_tensor_slices(X_train[[f]].values) for f in cat_features_names}
cat_features_dev = {f: tf.data.Dataset.from_tensor_slices(X_dev[[f]].values) for f in cat_features_names}

# Creating TensorFlow datasets for continuous features
cont_features_train = tf.data.Dataset.from_tensor_slices(X_train[cont_features_names].values)
cont_features_dev = tf.data.Dataset.from_tensor_slices(X_dev[cont_features_names].values)

# Creating TensorFlow datasets for labels
labels_train = tf.data.Dataset.from_tensor_slices(y_train.values)
labels_dev = tf.data.Dataset.from_tensor_slices(y_dev.values)

# Combine categorical, continuous features, and labels datasets
def combine_datasets(cat_datasets, cont_dataset, label_dataset):
    dataset = tf.data.Dataset.zip((*cat_datasets, cont_dataset, label_dataset))
    def map_func(*batches):
        cat_batches = batches[:-2]
        cont_batch = batches[-2]
        label_batch = batches[-1]
        return (cat_batches + (cont_batch,)), label_batch
    combined_dataset = dataset.map(map_func)
    return combined_dataset.batch(128)

train_dataset = combine_datasets(list(cat_features_train.values()), cont_features_train, labels_train)
dev_dataset = combine_datasets(list(cat_features_dev.values()), cont_features_dev, labels_dev)


In [11]:
def build_model(cat_features, cont_features):
    # Define input layers
    cat_inputs = [tf.keras.layers.Input(shape=(1,), name=f'cat{i}') for i in range(len(cat_features))]
    cont_inputs = tf.keras.layers.Input(shape=(len(cont_features),), name='cont_inputs')
    # cont_inputs_bn = tf.keras.layers.BatchNormalization()(cont_inputs)

    # Embedding layers for categorical inputs
    flat_embeddings = []
    for i, f in enumerate(cat_features):
        input_dim = int(cat_features_card[f])
        output_dim = int(min(64, round(1.6 * input_dim ** .56)))  # based on the fastai library
        embedding = tf.keras.layers.Embedding(input_dim=input_dim, output_dim=output_dim)(cat_inputs[i])
        embedding = tf.keras.layers.SpatialDropout1D(.3)(embedding)
        flat_embeddings.append(tf.keras.layers.Flatten()(embedding))

    concatenated_inputs = tf.keras.layers.Concatenate()(flat_embeddings + [cont_inputs])
    concatenated_inputs_bn = tf.keras.layers.BatchNormalization()(concatenated_inputs)

    x = tf.keras.layers.Dense(256, activation='mish')(concatenated_inputs_bn)
    # x = tf.keras.layers.Dropout(.3)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    for units in (128,):
        inp = tf.keras.layers.Concatenate()([x, concatenated_inputs_bn])
        x = tf.keras.layers.Dense(units=units, activation='mish')(inp)
        x = tf.keras.layers.Dropout(.3)(x)
        x = tf.keras.layers.BatchNormalization()(x)

    # Output layer
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return keras.Model(cat_inputs + [cont_inputs], outputs)

# cosine decay - for later.
# initially use a single learning rate
# for a small number of epochs
epochs = 4
callbacks = []


In [12]:
cat_features_card = {
    'Previously_Insured': 2,
    'Region_Code': len(df_train['Region_Code'].unique()),
    'Policy_Sales_Channel': len(df_train['Policy_Sales_Channel'].unique()),
    'Driving_License': 2,
    'Gender':2,
    'Vehicle_Age' :len(df_train['Vehicle_Age'].unique()),
    'Vehicle_Damage' : len(df_train['Vehicle_Damage'].unique()),

}


In [13]:
model = build_model(cat_features_names, cont_features_names)


In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(optimizer=optimizer,loss = 'binary_crossentropy', metrics =[ tf.keras.metrics.AUC(name='auc')])

In [15]:
model.compile(
        optimizer=keras.optimizers.AdamW(learning_rate=1E-4),
        loss='binary_crossentropy',
        metrics=['auc'])

In [ ]:
model.fit(train_dataset, validation_data=dev_dataset, epochs=epochs)


Epoch 1/4
   54/89432 ━━━━━━━━━━━━━━━━━━━━ 4:16 3ms/step - auc: 0.6147 - loss: 0.8054   

I0000 00:00:1721876949.954546     125 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


89432/89432 ━━━━━━━━━━━━━━━━━━━━ 276s 3ms/step - auc: 0.8292 - loss: 0.2943 - val_auc: 0.8596 - val_loss: 0.2625
Epoch 2/4
89432/89432 ━━━━━━━━━━━━━━━━━━━━ 248s 3ms/step - auc: 0.8564 - loss: 0.2663 - val_auc: 0.8600 - val_loss: 0.2620
Epoch 3/4
89432/89432 ━━━━━━━━━━━━━━━━━━━━ 249s 3ms/step - auc: 0.8576 - loss: 0.2654 - val_auc: 0.8606 - val_loss: 0.2616
Epoch 4/4
26882/89432 ━━━━━━━━━━━━━━━━━━━━ 2:55 3ms/step - auc: 0.8580 - loss: 0.2654

In [ ]:
cat_features_names = ['Previously_Insured', 'Region_Code', 'Policy_Sales_Channel', 'Gender_Female', 'Gender_Male', 'Vehicle_Age_1-2 Year', 'Vehicle_Age_< 1 Year', 'Vehicle_Age_> 2 Years', 'Vehicle_Damage_No', 'Vehicle_Damage_Yes']
cont_features_names = ['Age', 'Annual_Premium', 'Vintage']


# Creating TensorFlow datasets for categorical features in df_test
cat_features_test = {f: tf.data.Dataset.from_tensor_slices(df_test[[f]].values) for f in cat_features_names}

# Creating TensorFlow datasets for continuous features in df_test
cont_features_test = tf.data.Dataset.from_tensor_slices(df_test[cont_features_names].values)

# Combine categorical and continuous features datasets
def combine_test_datasets(cat_datasets, cont_dataset):
    dataset = tf.data.Dataset.zip((*cat_datasets, cont_dataset))
    def map_func(*batches):
        cat_batches = batches[:-1]
        cont_batch = batches[-1]
        return cat_batches + (cont_batch,)
    combined_dataset = dataset.map(map_func)
    return combined_dataset.batch(512)

test_dataset = combine_test_datasets(list(cat_features_test.values()), cont_features_test)

# Using the model for prediction
predictions = model.predict(test_dataset)

In [7]:
import tensorflow as tf

# Define custom GELU activation function
def gelu(x):
    return 0.5 * x * (1 + tf.math.erf(x / tf.sqrt(2.0)))

# Define input layer
input = tf.keras.Input(shape=(12,))

# Define the model architecture with GELU activation
x = tf.keras.layers.Dense(200, activation=gelu)(input)
x = tf.keras.layers.BatchNormalization()(x)  # Batch normalization
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(150, activation=gelu)(x)
x = tf.keras.layers.BatchNormalization()(x)  # Batch normalization
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Dense(100, activation=gelu)(x)
x = tf.keras.layers.BatchNormalization()(x)  # Batch normalization
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(50, activation=gelu)(x)
x = tf.keras.layers.BatchNormalization()(x)  # Batch normalization
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(25, activation=gelu)(x)
x = tf.keras.layers.BatchNormalization()(x)  # Batch normalization
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(15, activation=gelu)(x)
x = tf.keras.layers.BatchNormalization()(x)  # Batch normalization
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(5, activation=gelu)(x)
x = tf.keras.layers.BatchNormalization()(x)  # Batch normalization
x = tf.keras.layers.Dropout(0.2)(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create model
model = tf.keras.Model(inputs=input, outputs=outputs)

# Model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 200)            │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 150)            │           600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 50)             │           200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 25)             │           100 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 25)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 15)             │            60 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 15)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 5)              │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 5)              │            20 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 56,831 (222.00 KB)

 Trainable params: 55,741 (217.74 KB)

 Non-trainable params: 1,090 (4.26 KB)

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(optimizer=optimizer,loss = 'binary_crossentropy', metrics =[ tf.keras.metrics.AUC(name='auc')])

In [9]:
history = model.fit(train_ds, 
                    validation_data=vel_ds,
                    epochs=10)


NameError: name 'train_ds' is not defined

In [12]:
pred = model.predict(df_test, batch_size = 512)


14981/14981 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step


In [13]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e7/sample_submission.csv')
sub['Response'] = pred

In [14]:
sub.to_csv('submission4.csv', index = False)
